# Crear Target

Instalamos / actualizamos librerías

In [1]:
##%%bash 
# ! pip install duckdb
# ! pip install jupysql
# ! pip install duckdb-engine
# ! pip install pandas

In [4]:
# ! pip install --upgrade duckdb
# ! pip install --upgrade jupysql
# ! pip install --upgrade duckdb-engine

Configuramos el entorno de ejecución. Si ya tiene todo instalado, solo necesita ejecutar esta celda para empezar a usar **duckdb** 

In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/dmeyf.db

There's a new jupysql version available (0.10.3), you're running 0.10.1. To upgrade: pip install jupysql --upgrade


Cargamos datos 3era competencia:

In [2]:
%%sql
create or replace table competencia_03 as
select
    *
from read_csv_auto('C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/datasets/competencia_03.csv.gz')

,Success


In [3]:
%%sql
select
    *   
from competencia_03
limit 2

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,8,0.0,-35773.32,2.33,5148,36602.13,54,0,5008.71,CONTINUA
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,1,0.0,-67414.82,1149.57,3299,28503.06,10,0,1395.87,CONTINUA


Lista de variables con las que generaré nuevas variables:

In [4]:
# Output del boruta, excluyendo foto_mes, antiguedad_cliente
campos = ['active_quarter', 'mrentabilidad',
       'mrentabilidad_annual', 'mcomisiones', 'mactivos_margen',
       'mpasivos_margen', 'cproductos', 'mcuenta_corriente', 'mcaja_ahorro',
       'cdescubierto_preacordado', 'mcuentas_saldo',
       'ctarjeta_debito_transacciones', 'mautoservicio', 'ctarjeta_visa',
       'ctarjeta_visa_transacciones', 'mtarjeta_visa_consumo',
       'cprestamos_personales', 'mprestamos_personales', 'cpayroll_trx',
       'mpayroll', 'ccomisiones_mantenimiento', 'mcomisiones_mantenimiento',
       'ccomisiones_otras', 'mcomisiones_otras', 'ctransferencias_recibidas',
       'mtransferencias_recibidas', 'mextraccion_autoservicio',
       'chomebanking_transacciones', 'ctrx_quarter', 'Master_status',
       'Master_fechaalta', 'Visa_status', 'Visa_Fvencimiento',
       'Visa_msaldototal', 'Visa_msaldopesos', 'Visa_madelantopesos',
       'Visa_madelantodolares', 'Visa_fechaalta', 'Visa_cadelantosefectivo',
       'Visa_mpagominimo']
print(len(campos))

40


In [5]:
# campos
variables = ""
for campo in campos:
  variables += f"{campo},"
print(variables)

active_quarter,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,mcuenta_corriente,mcaja_ahorro,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,cprestamos_personales,mprestamos_personales,cpayroll_trx,mpayroll,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,ctransferencias_recibidas,mtransferencias_recibidas,mextraccion_autoservicio,chomebanking_transacciones,ctrx_quarter,Master_status,Master_fechaalta,Visa_status,Visa_Fvencimiento,Visa_msaldototal,Visa_msaldopesos,Visa_madelantopesos,Visa_madelantodolares,Visa_fechaalta,Visa_cadelantosefectivo,Visa_mpagominimo,


### LAG: Valor del mes anterior de cada columna
### Delta: Calculo la diferencia entre el valor del mes anterior y el presente para cada campo

In [10]:
# Genero la query para cada campo
q_lag = ""
for campo in campos:
  q_lag += f"\n ,lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}  \n ,lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo} \n ,lag({campo}, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_{campo}  \n ,lag({campo}, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_{campo}  \n,lag({campo}, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_{campo} \n, lag({campo}, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_{campo}  \n,  {campo} - lag_1_{campo} as delta_1_{campo}"
#print(q_lag)

In [11]:
with open("C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/q_lag_delta_c3.txt", "w") as archivo:
    archivo.write(q_lag)

In [12]:
%%sql
create or replace table competencia_03_lag_delta as
select
  numero_de_cliente
  ,foto_mes
  ,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,mpasivos_margen,cproductos,tcuentas,ccuenta_corriente,mcuenta_corriente_adicional,mcuenta_corriente,ccaja_ahorro,mcaja_ahorro,mcaja_ahorro_adicional,mcaja_ahorro_dolares,cdescubierto_preacordado,mcuentas_saldo,ctarjeta_debito,ctarjeta_debito_transacciones,mautoservicio,ctarjeta_visa,ctarjeta_visa_transacciones,mtarjeta_visa_consumo,ctarjeta_master,ctarjeta_master_transacciones,mtarjeta_master_consumo,cprestamos_personales,mprestamos_personales,cprestamos_prendarios,mprestamos_prendarios,cprestamos_hipotecarios,mprestamos_hipotecarios,cplazo_fijo,mplazo_fijo_dolares,mplazo_fijo_pesos,cinversion1,minversion1_pesos,minversion1_dolares,cinversion2,minversion2,cseguro_vida,cseguro_auto,cseguro_vivienda,cseguro_accidentes_personales,ccaja_seguridad,cpayroll_trx,mpayroll,mpayroll2,cpayroll2_trx,ccuenta_debitos_automaticos,mcuenta_debitos_automaticos,ctarjeta_visa_debitos_automaticos,mttarjeta_visa_debitos_automaticos,ctarjeta_master_debitos_automaticos,mttarjeta_master_debitos_automaticos,cpagodeservicios,mpagodeservicios,cpagomiscuentas,mpagomiscuentas,ccajeros_propios_descuentos,mcajeros_propios_descuentos,ctarjeta_visa_descuentos,mtarjeta_visa_descuentos,ctarjeta_master_descuentos,mtarjeta_master_descuentos,ccomisiones_mantenimiento,mcomisiones_mantenimiento,ccomisiones_otras,mcomisiones_otras,cforex,cforex_buy,mforex_buy,cforex_sell,mforex_sell,ctransferencias_recibidas,mtransferencias_recibidas,ctransferencias_emitidas,mtransferencias_emitidas,cextraccion_autoservicio,mextraccion_autoservicio,ccheques_depositados,mcheques_depositados,ccheques_emitidos,mcheques_emitidos,ccheques_depositados_rechazados,mcheques_depositados_rechazados,ccheques_emitidos_rechazados,mcheques_emitidos_rechazados,tcallcenter,ccallcenter_transacciones,thomebanking,chomebanking_transacciones,ccajas_transacciones,ccajas_consultas,ccajas_depositos,ccajas_extracciones,ccajas_otras,catm_trx,matm,catm_trx_other,matm_other,ctrx_quarter,Master_delinquency,Master_status,Master_mfinanciacion_limite,Master_Fvencimiento,Master_Finiciomora,Master_msaldototal,Master_msaldopesos,Master_msaldodolares,Master_mconsumospesos,Master_mconsumosdolares,Master_mlimitecompra,Master_madelantopesos,Master_madelantodolares,Master_fultimo_cierre,Master_mpagado,Master_mpagospesos,Master_mpagosdolares,Master_fechaalta,Master_mconsumototal,Master_cconsumos,Master_cadelantosefectivo,Master_mpagominimo,Visa_delinquency,Visa_status,Visa_mfinanciacion_limite,Visa_Fvencimiento,Visa_Finiciomora,Visa_msaldototal,Visa_msaldopesos,Visa_msaldodolares,Visa_mconsumospesos,Visa_mconsumosdolares,Visa_mlimitecompra,Visa_madelantopesos,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
 ,lag(active_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_active_quarter  
 ,lag(active_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_active_quarter 
 ,lag(active_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_active_quarter  
 ,lag(active_quarter, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_active_quarter  
,lag(active_quarter, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_active_quarter 
, lag(active_quarter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_active_quarter  
,  active_quarter - lag_1_active_quarter as delta_1_active_quarter
 ,lag(mrentabilidad, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad  
 ,lag(mrentabilidad, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mrentabilidad 
 ,lag(mrentabilidad, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad  
 ,lag(mrentabilidad, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mrentabilidad  
,lag(mrentabilidad, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mrentabilidad 
, lag(mrentabilidad, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mrentabilidad  
,  mrentabilidad - lag_1_mrentabilidad as delta_1_mrentabilidad
 ,lag(mrentabilidad_annual, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mrentabilidad_annual  
 ,lag(mrentabilidad_annual, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mrentabilidad_annual 
 ,lag(mrentabilidad_annual, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mrentabilidad_annual  
 ,lag(mrentabilidad_annual, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mrentabilidad_annual  
,lag(mrentabilidad_annual, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mrentabilidad_annual 
, lag(mrentabilidad_annual, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mrentabilidad_annual  
,  mrentabilidad_annual - lag_1_mrentabilidad_annual as delta_1_mrentabilidad_annual
 ,lag(mcomisiones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones  
 ,lag(mcomisiones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones 
 ,lag(mcomisiones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones  
 ,lag(mcomisiones, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcomisiones  
,lag(mcomisiones, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mcomisiones 
, lag(mcomisiones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones  
,  mcomisiones - lag_1_mcomisiones as delta_1_mcomisiones
 ,lag(mactivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mactivos_margen  
 ,lag(mactivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mactivos_margen 
 ,lag(mactivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mactivos_margen  
 ,lag(mactivos_margen, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mactivos_margen  
,lag(mactivos_margen, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mactivos_margen 
, lag(mactivos_margen, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mactivos_margen  
,  mactivos_margen - lag_1_mactivos_margen as delta_1_mactivos_margen
 ,lag(mpasivos_margen, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpasivos_margen  
 ,lag(mpasivos_margen, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpasivos_margen 
 ,lag(mpasivos_margen, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpasivos_margen  
 ,lag(mpasivos_margen, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mpasivos_margen  
,lag(mpasivos_margen, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mpasivos_margen 
, lag(mpasivos_margen, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpasivos_margen  
,  mpasivos_margen - lag_1_mpasivos_margen as delta_1_mpasivos_margen
 ,lag(cproductos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cproductos  
 ,lag(cproductos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cproductos 
 ,lag(cproductos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cproductos  
 ,lag(cproductos, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_cproductos  
,lag(cproductos, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_cproductos 
, lag(cproductos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cproductos  
,  cproductos - lag_1_cproductos as delta_1_cproductos
 ,lag(mcuenta_corriente, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuenta_corriente  
 ,lag(mcuenta_corriente, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuenta_corriente 
 ,lag(mcuenta_corriente, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuenta_corriente  
 ,lag(mcuenta_corriente, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcuenta_corriente  
,lag(mcuenta_corriente, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mcuenta_corriente 
, lag(mcuenta_corriente, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuenta_corriente  
,  mcuenta_corriente - lag_1_mcuenta_corriente as delta_1_mcuenta_corriente
 ,lag(mcaja_ahorro, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcaja_ahorro  
 ,lag(mcaja_ahorro, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcaja_ahorro 
 ,lag(mcaja_ahorro, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcaja_ahorro  
 ,lag(mcaja_ahorro, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcaja_ahorro  
,lag(mcaja_ahorro, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mcaja_ahorro 
, lag(mcaja_ahorro, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcaja_ahorro  
,  mcaja_ahorro - lag_1_mcaja_ahorro as delta_1_mcaja_ahorro
 ,lag(cdescubierto_preacordado, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cdescubierto_preacordado  
 ,lag(cdescubierto_preacordado, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cdescubierto_preacordado 
 ,lag(cdescubierto_preacordado, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cdescubierto_preacordado  
 ,lag(cdescubierto_preacordado, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_cdescubierto_preacordado  
,lag(cdescubierto_preacordado, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_cdescubierto_preacordado 
, lag(cdescubierto_preacordado, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cdescubierto_preacordado  
,  cdescubierto_preacordado - lag_1_cdescubierto_preacordado as delta_1_cdescubierto_preacordado
 ,lag(mcuentas_saldo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcuentas_saldo  
 ,lag(mcuentas_saldo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcuentas_saldo 
 ,lag(mcuentas_saldo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcuentas_saldo  
 ,lag(mcuentas_saldo, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcuentas_saldo  
,lag(mcuentas_saldo, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mcuentas_saldo 
, lag(mcuentas_saldo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcuentas_saldo  
,  mcuentas_saldo - lag_1_mcuentas_saldo as delta_1_mcuentas_saldo
 ,lag(ctarjeta_debito_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_debito_transacciones  
 ,lag(ctarjeta_debito_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_debito_transacciones 
 ,lag(ctarjeta_debito_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_debito_transacciones  
 ,lag(ctarjeta_debito_transacciones, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ctarjeta_debito_transacciones  
,lag(ctarjeta_debito_transacciones, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_ctarjeta_debito_transacciones 
, lag(ctarjeta_debito_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_debito_transacciones  
,  ctarjeta_debito_transacciones - lag_1_ctarjeta_debito_transacciones as delta_1_ctarjeta_debito_transacciones
 ,lag(mautoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mautoservicio  
 ,lag(mautoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mautoservicio 
 ,lag(mautoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mautoservicio  
 ,lag(mautoservicio, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mautoservicio  
,lag(mautoservicio, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mautoservicio 
, lag(mautoservicio, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mautoservicio  
,  mautoservicio - lag_1_mautoservicio as delta_1_mautoservicio
 ,lag(ctarjeta_visa, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa  
 ,lag(ctarjeta_visa, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa 
 ,lag(ctarjeta_visa, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa  
 ,lag(ctarjeta_visa, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ctarjeta_visa  
,lag(ctarjeta_visa, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_ctarjeta_visa 
, lag(ctarjeta_visa, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa  
,  ctarjeta_visa - lag_1_ctarjeta_visa as delta_1_ctarjeta_visa
 ,lag(ctarjeta_visa_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctarjeta_visa_transacciones  
 ,lag(ctarjeta_visa_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctarjeta_visa_transacciones 
 ,lag(ctarjeta_visa_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctarjeta_visa_transacciones  
 ,lag(ctarjeta_visa_transacciones, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ctarjeta_visa_transacciones  
,lag(ctarjeta_visa_transacciones, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_ctarjeta_visa_transacciones 
, lag(ctarjeta_visa_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctarjeta_visa_transacciones  
,  ctarjeta_visa_transacciones - lag_1_ctarjeta_visa_transacciones as delta_1_ctarjeta_visa_transacciones
 ,lag(mtarjeta_visa_consumo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtarjeta_visa_consumo  
 ,lag(mtarjeta_visa_consumo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtarjeta_visa_consumo 
 ,lag(mtarjeta_visa_consumo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtarjeta_visa_consumo  
 ,lag(mtarjeta_visa_consumo, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mtarjeta_visa_consumo  
,lag(mtarjeta_visa_consumo, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mtarjeta_visa_consumo 
, lag(mtarjeta_visa_consumo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtarjeta_visa_consumo  
,  mtarjeta_visa_consumo - lag_1_mtarjeta_visa_consumo as delta_1_mtarjeta_visa_consumo
 ,lag(cprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cprestamos_personales  
 ,lag(cprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cprestamos_personales 
 ,lag(cprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cprestamos_personales  
 ,lag(cprestamos_personales, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_cprestamos_personales  
,lag(cprestamos_personales, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_cprestamos_personales 
, lag(cprestamos_personales, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cprestamos_personales  
,  cprestamos_personales - lag_1_cprestamos_personales as delta_1_cprestamos_personales
 ,lag(mprestamos_personales, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mprestamos_personales  
 ,lag(mprestamos_personales, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mprestamos_personales 
 ,lag(mprestamos_personales, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mprestamos_personales  
 ,lag(mprestamos_personales, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mprestamos_personales  
,lag(mprestamos_personales, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mprestamos_personales 
, lag(mprestamos_personales, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mprestamos_personales  
,  mprestamos_personales - lag_1_mprestamos_personales as delta_1_mprestamos_personales
 ,lag(cpayroll_trx, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_cpayroll_trx  
 ,lag(cpayroll_trx, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_cpayroll_trx 
 ,lag(cpayroll_trx, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_cpayroll_trx  
 ,lag(cpayroll_trx, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_cpayroll_trx  
,lag(cpayroll_trx, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_cpayroll_trx 
, lag(cpayroll_trx, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_cpayroll_trx  
,  cpayroll_trx - lag_1_cpayroll_trx as delta_1_cpayroll_trx
 ,lag(mpayroll, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mpayroll  
 ,lag(mpayroll, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mpayroll 
 ,lag(mpayroll, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mpayroll  
 ,lag(mpayroll, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mpayroll  
,lag(mpayroll, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mpayroll 
, lag(mpayroll, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mpayroll  
,  mpayroll - lag_1_mpayroll as delta_1_mpayroll
 ,lag(ccomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_mantenimiento  
 ,lag(ccomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccomisiones_mantenimiento 
 ,lag(ccomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_mantenimiento  
 ,lag(ccomisiones_mantenimiento, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ccomisiones_mantenimiento  
,lag(ccomisiones_mantenimiento, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_ccomisiones_mantenimiento 
, lag(ccomisiones_mantenimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccomisiones_mantenimiento  
,  ccomisiones_mantenimiento - lag_1_ccomisiones_mantenimiento as delta_1_ccomisiones_mantenimiento
 ,lag(mcomisiones_mantenimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_mantenimiento  
 ,lag(mcomisiones_mantenimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_mantenimiento 
 ,lag(mcomisiones_mantenimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_mantenimiento  
 ,lag(mcomisiones_mantenimiento, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcomisiones_mantenimiento  
,lag(mcomisiones_mantenimiento, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mcomisiones_mantenimiento 
, lag(mcomisiones_mantenimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones_mantenimiento  
,  mcomisiones_mantenimiento - lag_1_mcomisiones_mantenimiento as delta_1_mcomisiones_mantenimiento
 ,lag(ccomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ccomisiones_otras  
 ,lag(ccomisiones_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ccomisiones_otras 
 ,lag(ccomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ccomisiones_otras  
 ,lag(ccomisiones_otras, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ccomisiones_otras  
,lag(ccomisiones_otras, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_ccomisiones_otras 
, lag(ccomisiones_otras, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ccomisiones_otras  
,  ccomisiones_otras - lag_1_ccomisiones_otras as delta_1_ccomisiones_otras
 ,lag(mcomisiones_otras, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mcomisiones_otras  
 ,lag(mcomisiones_otras, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mcomisiones_otras 
 ,lag(mcomisiones_otras, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mcomisiones_otras  
 ,lag(mcomisiones_otras, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mcomisiones_otras  
,lag(mcomisiones_otras, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mcomisiones_otras 
, lag(mcomisiones_otras, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mcomisiones_otras  
,  mcomisiones_otras - lag_1_mcomisiones_otras as delta_1_mcomisiones_otras
 ,lag(ctransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctransferencias_recibidas  
 ,lag(ctransferencias_recibidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctransferencias_recibidas 
 ,lag(ctransferencias_recibidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctransferencias_recibidas  
 ,lag(ctransferencias_recibidas, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ctransferencias_recibidas  
,lag(ctransferencias_recibidas, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_ctransferencias_recibidas 
, lag(ctransferencias_recibidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctransferencias_recibidas  
,  ctransferencias_recibidas - lag_1_ctransferencias_recibidas as delta_1_ctransferencias_recibidas
 ,lag(mtransferencias_recibidas, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mtransferencias_recibidas  
 ,lag(mtransferencias_recibidas, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mtransferencias_recibidas 
 ,lag(mtransferencias_recibidas, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mtransferencias_recibidas  
 ,lag(mtransferencias_recibidas, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mtransferencias_recibidas  
,lag(mtransferencias_recibidas, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mtransferencias_recibidas 
, lag(mtransferencias_recibidas, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mtransferencias_recibidas  
,  mtransferencias_recibidas - lag_1_mtransferencias_recibidas as delta_1_mtransferencias_recibidas
 ,lag(mextraccion_autoservicio, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_mextraccion_autoservicio  
 ,lag(mextraccion_autoservicio, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_mextraccion_autoservicio 
 ,lag(mextraccion_autoservicio, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_mextraccion_autoservicio  
 ,lag(mextraccion_autoservicio, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_mextraccion_autoservicio  
,lag(mextraccion_autoservicio, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_mextraccion_autoservicio 
, lag(mextraccion_autoservicio, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_mextraccion_autoservicio  
,  mextraccion_autoservicio - lag_1_mextraccion_autoservicio as delta_1_mextraccion_autoservicio
 ,lag(chomebanking_transacciones, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_chomebanking_transacciones  
 ,lag(chomebanking_transacciones, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_chomebanking_transacciones 
 ,lag(chomebanking_transacciones, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_chomebanking_transacciones  
 ,lag(chomebanking_transacciones, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_chomebanking_transacciones  
,lag(chomebanking_transacciones, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_chomebanking_transacciones 
, lag(chomebanking_transacciones, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_chomebanking_transacciones  
,  chomebanking_transacciones - lag_1_chomebanking_transacciones as delta_1_chomebanking_transacciones
 ,lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter  
 ,lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter 
 ,lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter  
 ,lag(ctrx_quarter, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_ctrx_quarter  
,lag(ctrx_quarter, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_ctrx_quarter 
, lag(ctrx_quarter, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_ctrx_quarter  
,  ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
 ,lag(Master_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_status  
 ,lag(Master_status, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_status 
 ,lag(Master_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_status  
 ,lag(Master_status, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Master_status  
,lag(Master_status, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Master_status 
, lag(Master_status, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_status  
,  Master_status - lag_1_Master_status as delta_1_Master_status
 ,lag(Master_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Master_fechaalta  
 ,lag(Master_fechaalta, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Master_fechaalta 
 ,lag(Master_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Master_fechaalta  
 ,lag(Master_fechaalta, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Master_fechaalta  
,lag(Master_fechaalta, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Master_fechaalta 
, lag(Master_fechaalta, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Master_fechaalta  
,  Master_fechaalta - lag_1_Master_fechaalta as delta_1_Master_fechaalta
 ,lag(Visa_status, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_status  
 ,lag(Visa_status, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_status 
 ,lag(Visa_status, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_status  
 ,lag(Visa_status, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_status  
,lag(Visa_status, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_status 
, lag(Visa_status, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_status  
,  Visa_status - lag_1_Visa_status as delta_1_Visa_status
 ,lag(Visa_Fvencimiento, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_Fvencimiento  
 ,lag(Visa_Fvencimiento, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_Fvencimiento 
 ,lag(Visa_Fvencimiento, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_Fvencimiento  
 ,lag(Visa_Fvencimiento, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_Fvencimiento  
,lag(Visa_Fvencimiento, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_Fvencimiento 
, lag(Visa_Fvencimiento, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_Fvencimiento  
,  Visa_Fvencimiento - lag_1_Visa_Fvencimiento as delta_1_Visa_Fvencimiento
 ,lag(Visa_msaldototal, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldototal  
 ,lag(Visa_msaldototal, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldototal 
 ,lag(Visa_msaldototal, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldototal  
 ,lag(Visa_msaldototal, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_msaldototal  
,lag(Visa_msaldototal, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_msaldototal 
, lag(Visa_msaldototal, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldototal  
,  Visa_msaldototal - lag_1_Visa_msaldototal as delta_1_Visa_msaldototal
 ,lag(Visa_msaldopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_msaldopesos  
 ,lag(Visa_msaldopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_msaldopesos 
 ,lag(Visa_msaldopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_msaldopesos  
 ,lag(Visa_msaldopesos, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_msaldopesos  
,lag(Visa_msaldopesos, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_msaldopesos 
, lag(Visa_msaldopesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_msaldopesos  
,  Visa_msaldopesos - lag_1_Visa_msaldopesos as delta_1_Visa_msaldopesos
 ,lag(Visa_madelantopesos, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantopesos  
 ,lag(Visa_madelantopesos, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_madelantopesos 
 ,lag(Visa_madelantopesos, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantopesos  
 ,lag(Visa_madelantopesos, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_madelantopesos  
,lag(Visa_madelantopesos, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_madelantopesos 
, lag(Visa_madelantopesos, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_madelantopesos  
,  Visa_madelantopesos - lag_1_Visa_madelantopesos as delta_1_Visa_madelantopesos
 ,lag(Visa_madelantodolares, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_madelantodolares  
 ,lag(Visa_madelantodolares, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_madelantodolares 
 ,lag(Visa_madelantodolares, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_madelantodolares  
 ,lag(Visa_madelantodolares, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_madelantodolares  
,lag(Visa_madelantodolares, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_madelantodolares 
, lag(Visa_madelantodolares, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_madelantodolares  
,  Visa_madelantodolares - lag_1_Visa_madelantodolares as delta_1_Visa_madelantodolares
 ,lag(Visa_fechaalta, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_fechaalta  
 ,lag(Visa_fechaalta, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_fechaalta 
 ,lag(Visa_fechaalta, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_fechaalta  
 ,lag(Visa_fechaalta, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_fechaalta  
,lag(Visa_fechaalta, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_fechaalta 
, lag(Visa_fechaalta, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_fechaalta  
,  Visa_fechaalta - lag_1_Visa_fechaalta as delta_1_Visa_fechaalta
 ,lag(Visa_cadelantosefectivo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_cadelantosefectivo  
 ,lag(Visa_cadelantosefectivo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_cadelantosefectivo 
 ,lag(Visa_cadelantosefectivo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_cadelantosefectivo  
 ,lag(Visa_cadelantosefectivo, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_cadelantosefectivo  
,lag(Visa_cadelantosefectivo, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_cadelantosefectivo 
, lag(Visa_cadelantosefectivo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_cadelantosefectivo  
,  Visa_cadelantosefectivo - lag_1_Visa_cadelantosefectivo as delta_1_Visa_cadelantosefectivo
 ,lag(Visa_mpagominimo, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_Visa_mpagominimo  
 ,lag(Visa_mpagominimo, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_Visa_mpagominimo 
 ,lag(Visa_mpagominimo, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_Visa_mpagominimo  
 ,lag(Visa_mpagominimo, 4) over (partition by numero_de_cliente order by foto_mes) as lag_4_Visa_mpagominimo  
,lag(Visa_mpagominimo, 5) over (partition by numero_de_cliente order by foto_mes) as lag_5_Visa_mpagominimo 
, lag(Visa_mpagominimo, 6) over (partition by numero_de_cliente order by foto_mes) as lag_6_Visa_mpagominimo  
,  Visa_mpagominimo - lag_1_Visa_mpagominimo as delta_1_Visa_mpagominimo

from competencia_03

,Success


### Promedio movil: 

In [13]:
# Genero la query para cada campo
q_avg = ""
for campo in campos:
  q_avg += f"\n ,avg({campo}) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_{campo}"
#print(q_avg)

In [14]:
with open("C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/q_avg_03.txt", "w") as archivo:
    archivo.write(q_avg)

In [15]:
%%sql
create or replace table competencia_03_avg as
select
  numero_de_cliente
  ,foto_mes
 ,avg(active_quarter) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_active_quarter
 ,avg(mrentabilidad) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mrentabilidad
 ,avg(mrentabilidad_annual) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mrentabilidad_annual
 ,avg(mcomisiones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcomisiones
 ,avg(mactivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mactivos_margen
 ,avg(mpasivos_margen) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mpasivos_margen
 ,avg(cproductos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cproductos
 ,avg(mcuenta_corriente) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcuenta_corriente
 ,avg(mcaja_ahorro) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcaja_ahorro
 ,avg(cdescubierto_preacordado) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cdescubierto_preacordado
 ,avg(mcuentas_saldo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcuentas_saldo
 ,avg(ctarjeta_debito_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_debito_transacciones
 ,avg(mautoservicio) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mautoservicio
 ,avg(ctarjeta_visa) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_visa
 ,avg(ctarjeta_visa_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctarjeta_visa_transacciones
 ,avg(mtarjeta_visa_consumo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtarjeta_visa_consumo
 ,avg(cprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cprestamos_personales
 ,avg(mprestamos_personales) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mprestamos_personales
 ,avg(cpayroll_trx) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_cpayroll_trx
 ,avg(mpayroll) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mpayroll
 ,avg(ccomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccomisiones_mantenimiento
 ,avg(mcomisiones_mantenimiento) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcomisiones_mantenimiento
 ,avg(ccomisiones_otras) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ccomisiones_otras
 ,avg(mcomisiones_otras) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mcomisiones_otras
 ,avg(ctransferencias_recibidas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctransferencias_recibidas
 ,avg(mtransferencias_recibidas) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mtransferencias_recibidas
 ,avg(mextraccion_autoservicio) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_mextraccion_autoservicio
 ,avg(chomebanking_transacciones) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_chomebanking_transacciones
 ,avg(ctrx_quarter) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_ctrx_quarter
 ,avg(Master_status) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_status
 ,avg(Master_fechaalta) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Master_fechaalta
 ,avg(Visa_status) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_status
 ,avg(Visa_Fvencimiento) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_Fvencimiento
 ,avg(Visa_msaldototal) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_msaldototal
 ,avg(Visa_msaldopesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_msaldopesos
 ,avg(Visa_madelantopesos) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_madelantopesos
 ,avg(Visa_madelantodolares) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_madelantodolares
 ,avg(Visa_fechaalta) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_fechaalta
 ,avg(Visa_cadelantosefectivo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_cadelantosefectivo
 ,avg(Visa_mpagominimo) over (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row) as avg_6_Visa_mpagominimo
 from competencia_03

,Success


### TENDENCIA:

In [16]:
# Genero la query para cada campo
q_reg = ""
for campo in campos:
  q_reg += f"\n ,regr_slope({campo}, cliente_antiguedad) over ventana_6 as {campo}_slope_6"
#print(q_reg)

In [17]:
with open("C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/q_reg_03.txt", "w") as archivo:
    archivo.write(q_reg)

In [18]:
%%sql
create or replace table competencia_03_reg as
SELECT 
    numero_de_cliente
    ,foto_mes
 ,regr_slope(active_quarter, cliente_antiguedad) over ventana_6 as active_quarter_slope_6
 ,regr_slope(mrentabilidad, cliente_antiguedad) over ventana_6 as mrentabilidad_slope_6
 ,regr_slope(mrentabilidad_annual, cliente_antiguedad) over ventana_6 as mrentabilidad_annual_slope_6
 ,regr_slope(mcomisiones, cliente_antiguedad) over ventana_6 as mcomisiones_slope_6
 ,regr_slope(mactivos_margen, cliente_antiguedad) over ventana_6 as mactivos_margen_slope_6
 ,regr_slope(mpasivos_margen, cliente_antiguedad) over ventana_6 as mpasivos_margen_slope_6
 ,regr_slope(cproductos, cliente_antiguedad) over ventana_6 as cproductos_slope_6
 ,regr_slope(mcuenta_corriente, cliente_antiguedad) over ventana_6 as mcuenta_corriente_slope_6
 ,regr_slope(mcaja_ahorro, cliente_antiguedad) over ventana_6 as mcaja_ahorro_slope_6
 ,regr_slope(cdescubierto_preacordado, cliente_antiguedad) over ventana_6 as cdescubierto_preacordado_slope_6
 ,regr_slope(mcuentas_saldo, cliente_antiguedad) over ventana_6 as mcuentas_saldo_slope_6
 ,regr_slope(ctarjeta_debito_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_debito_transacciones_slope_6
 ,regr_slope(mautoservicio, cliente_antiguedad) over ventana_6 as mautoservicio_slope_6
 ,regr_slope(ctarjeta_visa, cliente_antiguedad) over ventana_6 as ctarjeta_visa_slope_6
 ,regr_slope(ctarjeta_visa_transacciones, cliente_antiguedad) over ventana_6 as ctarjeta_visa_transacciones_slope_6
 ,regr_slope(mtarjeta_visa_consumo, cliente_antiguedad) over ventana_6 as mtarjeta_visa_consumo_slope_6
 ,regr_slope(cprestamos_personales, cliente_antiguedad) over ventana_6 as cprestamos_personales_slope_6
 ,regr_slope(mprestamos_personales, cliente_antiguedad) over ventana_6 as mprestamos_personales_slope_6
 ,regr_slope(cpayroll_trx, cliente_antiguedad) over ventana_6 as cpayroll_trx_slope_6
 ,regr_slope(mpayroll, cliente_antiguedad) over ventana_6 as mpayroll_slope_6
 ,regr_slope(ccomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as ccomisiones_mantenimiento_slope_6
 ,regr_slope(mcomisiones_mantenimiento, cliente_antiguedad) over ventana_6 as mcomisiones_mantenimiento_slope_6
 ,regr_slope(ccomisiones_otras, cliente_antiguedad) over ventana_6 as ccomisiones_otras_slope_6
 ,regr_slope(mcomisiones_otras, cliente_antiguedad) over ventana_6 as mcomisiones_otras_slope_6
 ,regr_slope(ctransferencias_recibidas, cliente_antiguedad) over ventana_6 as ctransferencias_recibidas_slope_6
 ,regr_slope(mtransferencias_recibidas, cliente_antiguedad) over ventana_6 as mtransferencias_recibidas_slope_6
 ,regr_slope(mextraccion_autoservicio, cliente_antiguedad) over ventana_6 as mextraccion_autoservicio_slope_6
 ,regr_slope(chomebanking_transacciones, cliente_antiguedad) over ventana_6 as chomebanking_transacciones_slope_6
 ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_6 as ctrx_quarter_slope_6
 ,regr_slope(Master_status, cliente_antiguedad) over ventana_6 as Master_status_slope_6
 ,regr_slope(Master_fechaalta, cliente_antiguedad) over ventana_6 as Master_fechaalta_slope_6
 ,regr_slope(Visa_status, cliente_antiguedad) over ventana_6 as Visa_status_slope_6
 ,regr_slope(Visa_Fvencimiento, cliente_antiguedad) over ventana_6 as Visa_Fvencimiento_slope_6
 ,regr_slope(Visa_msaldototal, cliente_antiguedad) over ventana_6 as Visa_msaldototal_slope_6
 ,regr_slope(Visa_msaldopesos, cliente_antiguedad) over ventana_6 as Visa_msaldopesos_slope_6
 ,regr_slope(Visa_madelantopesos, cliente_antiguedad) over ventana_6 as Visa_madelantopesos_slope_6
 ,regr_slope(Visa_madelantodolares, cliente_antiguedad) over ventana_6 as Visa_madelantodolares_slope_6
 ,regr_slope(Visa_fechaalta, cliente_antiguedad) over ventana_6 as Visa_fechaalta_slope_6
 ,regr_slope(Visa_cadelantosefectivo, cliente_antiguedad) over ventana_6 as Visa_cadelantosefectivo_slope_6
 ,regr_slope(Visa_mpagominimo, cliente_antiguedad) over ventana_6 as Visa_mpagominimo_slope_6
from competencia_03
window ventana_6 as (partition by numero_de_cliente order by foto_mes rows between 6 preceding and current row)

,Success


# Union tabla final:

In [19]:
%%sql
SELECT * FROM competencia_03_lag_delta 
LIMIT 2

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,lag_5_Visa_cadelantosefectivo,lag_6_Visa_cadelantosefectivo,delta_1_Visa_cadelantosefectivo,lag_1_Visa_mpagominimo,lag_2_Visa_mpagominimo,lag_3_Visa_mpagominimo,lag_4_Visa_mpagominimo,lag_5_Visa_mpagominimo,lag_6_Visa_mpagominimo,delta_1_Visa_mpagominimo
0,29207158,201901,1,0,1,58,238,2966.17,25845.45,1047.84,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29207158,201902,1,0,1,58,239,3875.19,28741.99,1379.24,...,NaN,NaN,0.0,293.25,NaN,NaN,NaN,NaN,NaN,903.21


In [20]:
%%sql
SELECT * FROM competencia_03_avg
LIMIT 2

,numero_de_cliente,foto_mes,avg_6_active_quarter,avg_6_mrentabilidad,avg_6_mrentabilidad_annual,avg_6_mcomisiones,avg_6_mactivos_margen,avg_6_mpasivos_margen,avg_6_cproductos,avg_6_mcuenta_corriente,...,avg_6_Master_fechaalta,avg_6_Visa_status,avg_6_Visa_Fvencimiento,avg_6_Visa_msaldototal,avg_6_Visa_msaldopesos,avg_6_Visa_madelantopesos,avg_6_Visa_madelantodolares,avg_6_Visa_fechaalta,avg_6_Visa_cadelantosefectivo,avg_6_Visa_mpagominimo
0,29246278,201901,1.0,12334.65,82529.30,2087.25,18.460,8717.600,9.0,0.0,...,4417.0,0.0,-2160.0,5659.95,6639.12,0.0,0.0,7357.0,0.0,480.93
1,29246278,201902,1.0,11941.81,85725.16,1706.54,57.775,8667.945,9.0,0.0,...,4431.0,0.0,-2146.0,5616.21,6587.81,0.0,0.0,7371.0,0.0,480.93


In [21]:
%%sql
SELECT * FROM competencia_03_reg
LIMIT 2

,numero_de_cliente,foto_mes,active_quarter_slope_6,mrentabilidad_slope_6,mrentabilidad_annual_slope_6,mcomisiones_slope_6,mactivos_margen_slope_6,mpasivos_margen_slope_6,cproductos_slope_6,mcuenta_corriente_slope_6,...,Master_fechaalta_slope_6,Visa_status_slope_6,Visa_Fvencimiento_slope_6,Visa_msaldototal_slope_6,Visa_msaldopesos_slope_6,Visa_madelantopesos_slope_6,Visa_madelantodolares_slope_6,Visa_fechaalta_slope_6,Visa_cadelantosefectivo_slope_6,Visa_mpagominimo_slope_6
0,29232333,202001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,29232333,202002,0.0,-14326.16,-14320.52,436.06,-75.66,-12509.36,0.0,-2267.51,...,29.0,0.0,29.0,-6904.53,-8099.01,NaN,NaN,29.0,NaN,0.0


In [22]:
%%sql
create or replace table competencia_03_fe as
SELECT * FROM competencia_03_lag_delta lag

LEFT JOIN competencia_03_avg avg 
ON lag.numero_de_cliente = avg.numero_de_cliente 
    AND lag.foto_mes = avg.foto_mes   
    
LEFT JOIN competencia_03_reg reg 
ON lag.numero_de_cliente = reg.numero_de_cliente 
    AND lag.foto_mes = reg.foto_mes     

,Success


In [29]:
%%sql
SELECT * FROM competencia_03_fe
LIMIT 2

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_status_slope_6,Visa_Fvencimiento_slope_6,Visa_msaldototal_slope_6,Visa_msaldopesos_slope_6,Visa_madelantopesos_slope_6,Visa_madelantodolares_slope_6,Visa_fechaalta_slope_6,Visa_cadelantosefectivo_slope_6,Visa_mpagominimo_slope_6,clase_ternaria
0,137211342,202107,1,0,0,46,64,2896.31,26304.38,1910.64,...,0.0,30.285714,-2288.075714,-2683.912857,0.0,0.0,30.285714,0.0,-365.305714,CONTINUA
1,137211342,202108,1,0,0,46,65,3266.20,28435.29,2228.13,...,0.0,30.607143,-1885.773214,-2212.012143,0.0,0.0,30.607143,0.0,-297.020357,NaN


Guardo un primer dataset con los meses de 202012 al 202107, (6 meses training, 1 test)

In [8]:
# %%sql
# create or replace table competencia_02_fe_202012 as
# SELECT * FROM competencia_02_fe
# WHERE foto_mes IN (202012,202101,202102,202103,202104,202105,202106,202107)

,Success


In [24]:
%%sql
create or replace table clase_ternaria as
SELECT numero_de_cliente, foto_mes, clase_ternaria FROM competencia_03

,Success


In [25]:
%%sql
create or replace table competencia_03_fe as
SELECT * FROM competencia_03_fe f
LEFT JOIN clase_ternaria p 
ON f.numero_de_cliente = p.numero_de_cliente 
    AND f.foto_mes = p.foto_mes  

,Success


In [27]:
%%sql
SELECT column_name
FROM information_schema.columns
WHERE table_name = 'competencia_03_fe' AND column_name LIKE '%numero_de_cliente%';

,column_name
0,numero_de_cliente
1,numero_de_cliente:1
2,numero_de_cliente:2
3,numero_de_cliente:3


In [28]:
%%sql
create or replace table competencia_03_fe as
SELECT * EXCLUDE ("numero_de_cliente:1", "numero_de_cliente:2",	"numero_de_cliente:3","foto_mes:1","foto_mes:2" ,"foto_mes:3") FROM competencia_03_fe

,Success


Guardamos el dataset final:

In [30]:
%%sql
copy competencia_03_fe to 'C:/Users/vanes/Documents/UBA/2do_cuatrimestre/DMEyF/datasets/competencia_03_fe.csv.gz' (FORMAT CSV, HEADER)

,Success
